In [ ]:
# -*- coding: utf-8 -*-

import math
from itertools import product

import numpy as np
from tqdm import tqdm
import micom
import pandas as pd




In [ ]:
def single_sl(model, cutoff, eli_list, solver):
    '''Analysis for single lethal reactions.'''
    model.solver = solver  # Basic solver configuration
    sol_wt = model.cooperative_tradeoff(fraction = 0.7, fluxes = True)  # Cooperative tradeoff - MICOM
    gr_wt = sol_wt.growth_rate

    solver_tol = model.solver.configuration.tolerances.optimality

    # Reformatting fluxes
    rxn_fluxes = {}
    for i in sol_wt.fluxes.columns:
        rxn_fluxes[str(i) + '__model_1'] = sol_wt.fluxes[str(i)]['model_1']
        rxn_fluxes[str(i) + '__model_2'] = sol_wt.fluxes[str(i)]['model_2']

    rxn_fluxes = pd.Series(rxn_fluxes).dropna()

    # Indices of non-zero flux reactions including exchange/diffusion reactions
    jnz_before_filtering = np.where(rxn_fluxes.abs() > solver_tol)[0]
    '''
    print('Before filtering', len(jnz_before_filtering))
    for i in jnz_before_filtering:
        print(model.reactions[i].id)
    '''
    
    # Indices of exchange/diffusion reactions (not considered)
    eli_idx = [model.reactions.index(reaction_id) for reaction_id in eli_list]
    
    jnz = np.setdiff1d(jnz_before_filtering, eli_idx)  # jnz
    
    # Identify Single Lethal Reaction Deletions

    jsl_idx = []

    for del_idx_i in tqdm(jnz, desc="Identifying jsl reactions"):
        with model:
            model.reactions[del_idx_i].knock_out()
            try:
                sol_del = (model.cooperative_tradeoff(fraction = 0.7))
                sol_ko_i = sol_del.growth_rate
                sol_del_df = sol_del.members
                if sol_del.growth_rate <= 0.1 * sol_WT.growth_rate or sol_del_df._get_value('model_1', 'growth_rate') <= sol_del.growth_rate * 0.1 or  sol_del_df._get_value('model_2', 'growth_rate') <= sol_del.growth_rate * 0.1:
                    '''
                    if sol_ko_i < cutoff * gr_wt or math.isnan(sol_ko_i) is True:
                    '''
                    jsl_idx.append(int(del_idx_i))
                    
            except ValueError:
                print('Error', model.reactions[del_idx_i].id)
                pass

    # Indices -> Reactions
    jsl = model.reactions.get_by_any(jsl_idx)
    # jsl = [model.reactions[rxn_idx].id for rxn_idx in jsl_idx]

    return jsl


def double_sl(model, cutoff, eli_list, solver):
    '''Analysis for double lethal reactions.'''
    model.solver = solver  # Basic solver configuration
    sol_wt = model.cooperative_tradeoff(fraction = 0.7, fluxes = True)  # Identify minNorm flux distribution
    gr_wt = sol_wt.growth_rate

    solver_tol = model.solver.configuration.tolerances.optimality

    # Reformatting fluxes
    rxn_fluxes = {}
    for i in sol_wt.fluxes.columns:
        rxn_fluxes[str(i) + '__model_1'] = sol_wt.fluxes[str(i)]['model_1']
        rxn_fluxes[str(i) + '__model_2'] = sol_wt.fluxes[str(i)]['model_2']

    rxn_fluxes = pd.Series(rxn_fluxes).dropna()
    
    # Indices of non-zero flux reactions including exchange/diffusion reactions
    jnz_before_filtering = np.where(rxn_fluxes.abs() > solver_tol)[0]

    # Indices of exchange/diffusion reactions (not considered)
    eli_idx = [model.reactions.index(reaction_id) for reaction_id in eli_list]

    jnz = np.setdiff1d(jnz_before_filtering, eli_idx)  # jnz

    # Identify single lethal reactions
    jsl = single_sl(model,
                    cutoff,
                    eli_list,
                    solver)

    # Indices of single lethal reacions
    jsl_idx = [model.reactions.index(jsl_id) for jsl_id in jsl]

    jnz_copy = np.setdiff1d(jnz, jsl_idx)  # jnz-jsl

    # Makes rxn pairs to remove nested for-loops in phase 2
    jnz_copy_phase_2 = [rxn_pair for rxn_pair in product(jnz_copy, repeat=2)]

    # Identify double lethal reactions

    jdl_idx = []

    # Phase 1:
    for del_idx_i in tqdm(jnz_copy, desc="Identifying jdl reactions: 1 of 2"):
        with model:
            model.reactions[del_idx_i].knock_out()
            sol_ko_i = model.cooperative_tradeoff(fraction = 0.7, fluxes = True)
            
            # Reformatting fluxes
            rxn_fluxes_ko_i = {}
            for i in sol_ko_i.fluxes.columns:
                rxn_fluxes_ko_i[str(i) + '__model_1'] = sol_ko_i.fluxes[str(i)]['model_1']
                rxn_fluxes_ko_i[str(i) + '__model_2'] = sol_ko_i.fluxes[str(i)]['model_2']

            rxn_fluxes_ko_i = pd.Series(rxn_fluxes_ko_i).dropna()            
            
            newnnz = np.where(rxn_fluxes_ko_i.abs() > solver_tol)[0]
            
            jnz_i_before_filtering = np.setdiff1d(newnnz, jnz)
            jnz_i = np.setdiff1d(jnz_i_before_filtering, eli_idx)

            for del_idx_j in jnz_i:
                with model:
                    model.reactions[del_idx_j].knock_out()
                    try:
                        sol_ko_ij = model.cooperative_tradeoff(fraction = 0.7, fluxes = True)

                        # Reformatting fluxes
                        rxn_fluxes_ko_ij = {}
                        for i in sol_ko_ij.fluxes.columns:
                            rxn_fluxes_ko_ij[str(i) + '__model_1'] = sol_ko_ij.fluxes[str(i)]['model_1']
                            rxn_fluxes_ko_ij[str(i) + '__model_2'] = sol_ko_ij.fluxes[str(i)]['model_2']

                        rxn_fluxes_ko_ij = pd.Series(rxn_fluxes_ko_ij).dropna()    

                        if sol_ko_ij.growth_rate < cutoff * gr_wt or \
                                math.isnan(sol_ko_ij.growth_rate) is True:
                            jdl_idx.append([int(del_idx_i), int(del_idx_j)])
                    except:
                        print('Error')

    # Phase 2:
    for del_idx_pair in tqdm(jnz_copy_phase_2,
                             desc="Identifying jdl reactions: 2 of 2"):
        del_idx_i, del_idx_j = del_idx_pair
        if np.where(jnz_copy == del_idx_j) < np.where(jnz_copy == del_idx_i):
            with model:
                model.reactions[del_idx_i].knock_out()
                model.reactions[del_idx_j].knock_out()
                
                try:
                    sol_del = (model.cooperative_tradeoff(fraction = 0.7)).growth_rate
                    sol_del_df = sol_del.members

                    if sol_del.growth_rate <= 0.1 * sol_WT.growth_rate or sol_del_df._get_value('model_1', 'growth_rate') <= sol_del.growth_rate * 0.1 or  sol_del_df._get_value('model_2', 'growth_rate') <= sol_del.growth_rate * 0.1:
                        '''   
                        if sol_del < cutoff * gr_wt or \
                                math.isnan(sol_ko_ij) is True:
                            jdl_idx.append([int(del_idx_i), int(del_idx_j)])
                        ''' 
                except:
                    print('Error')
    # Indices -> Reactions
    jdl = [model.reactions.get_by_any(rxn_pair_idx) for rxn_pair_idx
           in jdl_idx]

    return (jsl, jdl)

In [ ]:
from cobra.io import read_sbml_model
from cobra.medium import minimal_medium
from micom import Community

In [ ]:
org_list_file = open('Models.csv')
org_names = [(i.split())[0] for i in org_list_file.readlines()]

pairwise_orgs = []
for i in range(len(org_names)):
    for j in range(i + 1, len(org_names)):
        pairwise_orgs.append(org_names[i] + ',' + org_names[j])

In [ ]:
pair = pairwise_orgs[0]
orgs = pair.split(',')
file_1 = f'Models/{orgs[0]}.xml'
file_2 = f'Models/{orgs[1]}.xml'

In [ ]:
model_1 = read_sbml_model(file_1)
model_2 = read_sbml_model(file_2)

table = {'id': ['model_1', 'model_2'], 'file': [file_1, file_2]}
data = pd.DataFrame(table)
taxonomy = data

com = Community(taxonomy, progress=False)
com.medium = minimal_medium(com, 1)


In [ ]:
sol_WT = com.cooperative_tradeoff(fraction=0.7, fluxes=True)

In [ ]:
print(sol_WT.fluxes)

In [ ]:
results = double_sl(com, 0.1, [], 'gurobi')